In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE129604"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE129604"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE129604.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE129604.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE129604.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains whole blood gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait - Vitamin D Levels
# From the sample characteristics, we can see agent row (2) contains information about vitamin D supplementation
trait_row = 2

# For age
# Age information is not available in the sample characteristics
age_row = None

# For gender
# Gender information is available in the sample characteristics at row 0
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert vitamin D treatment information to binary"""
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Create binary classification: 1 for vitamin D treatment, 0 for non-vitamin D treatment
    if 'VitD' in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age to continuous numeric value"""
    # No age data available
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability based on trait_row
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the matrix file line by line to extract the sample characteristics section correctly
    sample_data = []
    in_characteristics = False
    
    with gzip.open(f"{in_cohort_dir}/GSE129604_series_matrix.txt.gz", 'rt') as f:
        for line in f:
            line = line.strip()
            if line.startswith('!Sample_characteristics_ch1'):
                in_characteristics = True
                char_value = line.replace('!Sample_characteristics_ch1', '').strip()
                sample_data.append(char_value)
            elif in_characteristics and line.startswith('!'):
                if not line.startswith('!Sample_characteristics_ch1'):
                    in_characteristics = False
    
    # Determine the number of samples
    num_samples = len(sample_data)
    
    # Group the characteristics by row
    grouped_chars = {}
    row_index = 0
    
    for i in range(0, num_samples, 1):
        if i < len(sample_data):
            char_value = sample_data[i]
            if row_index not in grouped_chars:
                grouped_chars[row_index] = []
            grouped_chars[row_index].append(char_value)
        
        if (i + 1) % 4 == 0:  # Each sample has 4 characteristics
            row_index += 1
    
    # Create a DataFrame from the grouped characteristics
    clinical_data = pd.DataFrame(grouped_chars)
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the previous step
# The identifiers (AFFX-BkGr-GC03_st, etc.) are Affymetrix probe IDs from a microarray platform
# They are not standard human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 6: Gene Identifier Mapping

In [ ]:
# Extract gene symbols from the SPOT_ID.1 column, which contains detailed annotation including gene symbols
def extract_gene_symbol(annotation_str):
    """Extract gene symbols from complex annotation strings in SPOT_ID.1 column"""
    if not isinstance(annotation_str, str):
        return []
    
    # Look for gene symbols in the format [Source:HGNC Symbol;Acc:HGNC:xxxx]
    hgnc_pattern = r'\[Source:HGNC Symbol;Acc:HGNC:\d+\]'
    
    # Find all occurrences that match the pattern
    matches = re.findall(hgnc_pattern, annotation_str)
    
    # Get the words right before each HGNC reference, which should be the gene names
    gene_names = []
    for match in matches:
        # Find where in the original string this match occurs
        start_idx = annotation_str.find(match)
        if start_idx > 0:
            # Look for the word before the match
            before_text = annotation_str[:start_idx].strip()
            words = before_text.split()
            if words:
                gene_name = words[-1]
                # Clean any non-alphanumeric characters except certain allowed ones
                gene_name = re.sub(r'[^A-Za-z0-9\-]', '', gene_name)
                if gene_name:
                    gene_names.append(gene_name)
    
    # If no HGNC symbols found, try to extract gene symbols from RefSeq entries
    if not gene_names:
        refseq_pattern = r'NM_\d+ // RefSeq // Homo sapiens ([^(]+)'
        refseq_matches = re.findall(refseq_pattern, annotation_str)
        for match in refseq_matches:
            gene_name = match.split('(')[0].strip()
            if ',' in gene_name:
                gene_name = gene_name.split(',')[0].strip()
            if gene_name:
                gene_names.append(gene_name)
    
    # Deduplicate gene names
    return list(set(gene_names))

# Add gene symbols to the annotation dataframe
gene_annotation['Gene_Symbols'] = gene_annotation['SPOT_ID.1'].apply(extract_gene_symbol)

# Check the IDs in gene expression data
print("Sample IDs from gene expression data:")
print(gene_data.index[:5])

# Check if there are matching IDs in the annotation
matching_ids = [idx for idx in gene_data.index if idx in gene_annotation['ID'].values]
print(f"\nNumber of IDs from gene expression data that match annotation: {len(matching_ids)}")

# If there's a mismatch, analyze the format of IDs in both datasets
if len(matching_ids) < 100:
    # Looking for patterns in the gene expression IDs
    print("\nPattern in gene expression IDs:")
    expression_id_pattern = re.findall(r'([A-Za-z\-]+)(\d+)', gene_data.index[0])
    print(f"Expression ID pattern example: {expression_id_pattern}")
    
    # Looking for patterns in the annotation IDs
    print("\nPattern in annotation IDs:")
    annotation_id_pattern = re.findall(r'([A-Za-z\-]+)(\d+)', gene_annotation['ID'].iloc[0])
    print(f"Annotation ID pattern example: {annotation_id_pattern}")

# Let's check for a different IDs that might match between datasets
print("\nChecking for alternative ID matches...")

# Get a sample of probe IDs from gene_annotation
sample_annotation_ids = gene_annotation['probeset_id'].head(10).tolist()
print("Sample annotation probeset_ids:", sample_annotation_ids)

# Check if any of these exist in the gene expression data
found_in_expression = [id in gene_data.index for id in sample_annotation_ids]
print(f"Found in expression data: {sum(found_in_expression)} out of 10")

# The probeset_id seems to be a better match for what we need for mapping
# Create a mapping dataframe with probeset_id and Gene_Symbols
mapping_df = pd.DataFrame({
    'ID': gene_annotation['probeset_id'],
    'Gene': gene_annotation['Gene_Symbols']
})

# Remove rows with empty gene symbols
mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0)]
print(f"\nCreated mapping with {len(mapping_df)} entries")

# Apply gene mapping to convert probe measurements to gene expression
try:
    # First explode the Gene column to handle one-to-many mappings
    gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
    print(f"\nConverted probe data to gene expression data with {len(gene_data_mapped)} genes")
    
    # Check the first few genes
    print("\nFirst 10 genes in the mapped data:")
    print(gene_data_mapped.index[:10])
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_mapped.to_csv(out_gene_data_file)
    print(f"\nGene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    
    # Let's try a simpler approach if the mapping fails
    print("\nAttempting alternative mapping approach...")
    
    # Let's attempt to extract gene symbols from the SOFT file directly
    gene_symbols = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if "gene_assignment" in line.lower() and "=" in line:
                parts = line.split("=")
                if len(parts) > 1:
                    gene_info = parts[1].strip()
                    print(f"Sample gene assignment: {gene_info}")
                    break
    
    # If we can't get a proper mapping, let's normalize the dataset using the extract_human_gene_symbols function
    # By processing each probeset ID in the SPOT_ID.1 column
    print("\nPerforming direct gene symbol extraction from annotation...")
    mapping_df = pd.DataFrame({
        'ID': gene_annotation['ID'],
        'Gene': gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
    })
    
    # Remove rows with empty gene symbols
    mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0)]
    print(f"Created mapping with {len(mapping_df)} entries using direct gene symbol extraction")
    
    # Apply gene mapping to convert probe measurements to gene expression
    gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
    print(f"Converted probe data to gene expression data with {len(gene_data_mapped)} genes")
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_mapped.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")

# Assign the mapped data to gene_data for the next steps
gene_data = gene_data_mapped